<font face="微軟正黑體" size=5>1. Dump files</font><br>

In [3]:
#1. Dump files

#dump langlinks file
#!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-langlinks.sql.gz

#dump file mapping pageid to page title
#!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-page.sql.gz

#dump redirect file
#!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-redirect.sql.gz

#dump wiki article
#!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream.xml.bz2


<font face="微軟正黑體" size=5>2. 找出英文pageid 與title的對應與中英文對應的link</font><br>
<font face="微軟正黑體" size=5>output1：en.page.id.txt; 形式： page_id \t en_title</font><br>

In [20]:
%%bash
# 2. 找出英文pageid 與title的對應與中英文對應的link
# output1：en.page.id.txt; 形式： page_id \t en_title

zgrep -P -o "\([0-9]+?,0,'[^'].+?'" enwiki-latest-page.sql.gz | sed -e 's/^(//' | sed -e "s/,0,/\t/g" | sort -t $'\t' -k1,1 > temp_pageid_strorder.txt
cat temp_pageid_strorder.txt | sort -n -t $'\t' -k1,1 > en.page.id.txt


In [21]:
!head -5 en.page.id.txt

10	'AccessibleComputing'
12	'Anarchism'
13	'AfghanistanHistory'
14	'AfghanistanGeography'
15	'AfghanistanPeople'


In [22]:
!grep "Senior_(education)" en.page.id.txt

24069615	'Senior_(education)'


<font face="微軟正黑體" size=5>output2：ec.link.txt; 形式： page_id \t en_title \t zh_title</font>

In [23]:
%%bash

#output2：ec.link.txt; 形式： page_id \t en_title \t zh_title
#有些項目grep會切錯，所以grep做兩次
zgrep -P -o "\([0-9]+?,'zh','[^'].+?'\)" enwiki-latest-langlinks.sql.gz > temp0.txt
grep -P -o "\([0-9]+?,'zh','.+?'\)" temp0.txt | sort -t, -k1,1 > temp1_s.txt
grep -P -o "[0-9]+?,'zh','.+?'" temp1_s.txt | sed -e "s/,'zh',/\t/" | sort -t $'\t' -k1,1 > temp1_tab.txt
join -t $'\t' temp_pageid_strorder.txt temp1_tab.txt | sort -n -t $'\t' -k1,1 > ec.link.txt


In [24]:
!head ec.link.txt

12	'Anarchism'	'无政府主义'
25	'Autism'	'自閉症'
39	'Albedo'	'反照率'
290	'A','	'A'
303	'Alabama'	'亚拉巴马州'
305	'Achilles'	'阿喀琉斯'
307	'Abraham_Lincoln'	'亚伯拉罕·林肯'
308	'Aristotle'	'亚里士多德'
309	'An_American_in_Paris'	'一個美國人在巴黎'
316	'Academy_Award_for_Best_Production_Design'	'奥斯卡最佳艺术指导奖'


<font face="微軟正黑體" size=5>3. 找出redirect page</font><br>
<font face="微軟正黑體" size=5>檔名：en.redirect.txt; 形式： from_page_id \t from_en_title \t to_en_title</font>

In [28]:
%%bash
#3. 找出redirect page
#檔名：en.redirect.txt; 形式： from_page_id \t from_en_title \t to_en_title
zgrep -P -o "[0-9]+?,0,'[^'].+?'" enwiki-latest-redirect.sql.gz | sort -t, -k1,1 | sed -e "s/,0,/\t/g" > temp_re_tab.txt
join -t $'\t' temp_pageid_strorder.txt temp_re_tab.txt | sort -n -t $'\t' -k1,1 > en.redirect.txt


In [29]:
!grep "'Plant'" ec.link.txt

19828134	'Plant'	'植物'


In [31]:
!head en.redirect.txt

10	'AccessibleComputing'	'Computer_accessibility'
13	'AfghanistanHistory'	'History_of_Afghanistan'
14	'AfghanistanGeography'	'Geography_of_Afghanistan'
15	'AfghanistanPeople'	'Demographics_of_Afghanistan'
18	'AfghanistanCommunications'	'Communications_in_Afghanistan'
19	'AfghanistanTransportations'	'Transport_in_Afghanistan'
20	'AfghanistanMilitary'	'Afghan_Armed_Forces'
21	'AfghanistanTransnationalIssues'	'Foreign_relations_of_Afghanistan'
23	'AssistiveTechnology'	'Assistive_technology'
24	'AmoeboidTaxa'	'Amoeba'


<font face="微軟正黑體" size=5>4. Count parentheses (for category、BRT...)</font><br>
<font face="微軟正黑體" size=5>output：en.link.pe.txt; format：count \t (word)</font><br>
<font face="微軟正黑體" size=5>output：en.link.txt_pe.txt; format：count \t text (word)</font><br>

In [ ]:
%%bash
#4. Count parentheses (for category、BRT...)
#output：en.link.pe.txt; format：count \t (word)
#output：en.link.txt_pe.txt; format：count \t text (word)

bzgrep -o '\[\[[^]#:][^]:]*\]\]' enwiki-latest-pages-articles-multistream.xml.bz2 | sed "s/#.*|/|/g" | sed "s/#.*]]/]]/g" | sort | gzip > en.link.anchor.gz
zgrep -P -o '\([^)]*?\)' en.link.anchor.gz | sort | uniq -c | sort -k1nr > en.link.pe.txt
zgrep -P -o '\[\[[^|]*?\)(\]|\|)' en.link.anchor.gz | sed "s/^..//" | sed "s/.$//" | sort | uniq -c | sort -k1nr > en.link.txt_pe.txt

In [5]:
!grep '(plant)' en.link.pe.txt

  14967 (plant)


In [6]:
! grep '(plant)' en.link.txt_pe.txt | head

    881 Iris (plant)
    385 Cutting (plant)
    341 Narcissus (plant)
    278 Squash (plant)
    273 Viola (plant)
    221 Morus (plant)
    203 Canna (plant)
    194 Reed (plant)
    171 Croton (plant)
    134 iris (plant)
grep: write error: Broken pipe


<font face="微軟正黑體" size=5>5. Redirect Link</font><br>
<font face="微軟正黑體" size=5>output：en.link.redirect.txt</font><br>
<font face="微軟正黑體" size=5>format：wn_lemma \t redirect_page_title \t zh_title \t sum(count_#) \t Count({'[link_1|anchor_1]':count_1 , ... })</font><br>

In [ ]:
#5.找出wiki page中所有內部超連結並統計次數[[link|anchor]]
!time bzgrep -o '\[\[[^]:#][^]:][^]:]*\]\]' enwiki-latest-pages-articles-multistream.xml.bz2 | sed "s/#.*|/|/g" | sed "s/#.*]]/]]/g" | sort | uniq -c | sort -nr | sed -e "s/^ *//" | gzip > en.link.gz

In [41]:
import re, sys, os
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from collections import defaultdict, Counter
import gzip

lemmatizer = WordNetLemmatizer()

def is_in_wn(word):
    if wn.synsets(word): return True
    if wn.synsets(lemmatizer.lemmatize(word, 'n')): return True
    return False

def clean(text):
    return re.sub(re.compile('\(.*?\)'), '', text).lstrip().strip().replace(' ', '_')



In [8]:
#找出所有在wordnet中有定義的anchor
# Step 1: time zcat en.link.gz | python wiki.sense.py | gzip > en.link.wn.gz
fp = open("en.link.wn", "w") #("link.anchor.cam.txt", "a")
for line in gzip.open('en.link.gz','rt').readlines(): # en.link.gz > link.anchor.cam.txt (303810)
    line = line.lstrip().strip()
    start = line.index(' ')
    count, linkanchor = line[:start], line[start+1:]
    anchor = linkanchor.split('|')[1][:-2] if '|' in linkanchor else linkanchor[2:-2]
    link = linkanchor.split('|')[0][2:] if '|' in linkanchor else linkanchor[2:-2]
    if len(anchor) == 0:  #note: some anchors are empty string
        continue
    if (not anchor[0].isupper()) and is_in_wn(anchor) and wn.synsets(clean(link)):
        fp.write('\t'.join( [linkanchor, lemmatizer.lemmatize(anchor, 'n'), count, '\n']) )
            #print ('\t'.join( [linkanchor, lemmatizer.lemmatize(anchor, 'n'), count]) )
fp.close()

In [1]:
!zgrep -o "\[\[[^]]*|\]\]" en.link.gz | head # note: empty source in anchor

[[Number sign|]]
[[Richmond Shipyards|]]
[[Me Too movement|]]
[[Notre dame football|]]
[[Oklahoma Sooners football|]]
[[Army Black Knights football|]]
[[Number 1's (Mariah Carey album)|]]
[[Me Too (hashtag)|]]
[[willpower (will.i.am album)|]]
[[chart-topper|]]
/bin/grep: write error: Broken pipe

gzip: stdout: Broken pipe


In [10]:
!gzip en.link.wn

In [9]:
!zcat en.link.wn.gz | head

[[moth]]	moth	80482	
[[village]]	village	64800	
[[species]]	specie	55654	
[[census]]	census	54188	
[[genus]]	genus	48168	
[[Association football|footballer]]	footballer	46704	
[[family (biology)|family]]	family	39782	
[[basketball]]	basketball	39640	
[[association football|footballer]]	footballer	38523	
[[actor]]	actor	36875	

gzip: stdout: Broken pipe


In [45]:
pageid={}
for line in open('en.page.id.txt').readlines():
    if '\t' in line:
        try:
            pid, title = line.lstrip().strip().split('\t')
            title = title.replace('\'', '')
            pageid[title] = pid 
        except:
            pass
        
def title_to_pageid(title):
    title = title.replace(' ', '_')
    if title in pageid:
        return pageid[title]
    else:
        return '0'



redirect, recase, = {}, {} # Party of the first part	Party_(law)
for line in open('en.redirect.txt').readlines():
    try:
        line = line.replace('\'','')
        from_id, mention, page =line.lstrip().strip().split('\t')
        mention = mention.replace('_', ' ')
        if mention.isupper() == True:
            #print(mention)
            redirect[mention] = page.replace('_', ' ')
        else:    
            redirect[mention.lower()] = page.replace('_', ' ')
        page = page.replace('_', ' ')
        recase[page.lower()] = page        
    except:
        pass    

    


In [46]:
def redirect_link(link):
    if link in redirect:
        return redirect[link]
    elif link.lower() in redirect:
        return redirect[link.lower()]
    elif link.lower() in recase:
        return recase[link.lower()]
    elif link.capitalize().replace(' ', '_') in pageid:
        #print(link.capitalize())
        return link.capitalize()
    else:
        return link

In [47]:
#列出超連結中link對應的redirect page
# Step 2: time zcat en.link.wn.gz | python wiki.sense.py | gzip > en.link.redirect.gz
lemma_page = defaultdict(lambda: defaultdict(lambda: Counter()))

for line in gzip.open('en.link.wn.gz','rt').readlines():  # en.link.wn.gz > en.link.redirect.gz 
    linkanchor, lemma, times = line.lstrip().strip().split('\t') # party	[[party (law)|party]]	23
    anchor = linkanchor.split('|')[1][:-2] if '|' in linkanchor else linkanchor[2:-2]
    link = linkanchor.split('|')[0][2:] if '|' in linkanchor else linkanchor[2:-2]
    lemma_page[lemma][redirect_link(link)][linkanchor] += int(times)

list1 = [ (lemma, page, lemma_page[lemma][page], sum(lemma_page[lemma][page].values())) for lemma in lemma_page for page in lemma_page[lemma] ]
list1.sort(key=lambda x: (x[0].lower(), -x[3]))
# 12	Anarchism	->無政府主義
langlink = {}
for line in open('ec.link.txt').readlines():
    try:
        line = line.replace('\'','')
        e, c  = line.strip().replace('->', '').split('\t')[1:]
        langlink[e.replace('_', ' ')] = c
    except:
        pass


In [36]:
#truecase_lemma_page = defaultdict(lambda: defaultdict(lambda: Counter())) 
fp_l = open("en.link.redirect.txt", "w")
for lemma, page, links, total in list1:
    '''if page in truecase:
        oldlemma, oldcount = truecase[page]
        if total > oldcount:
            truecase[page] = (lemma, oldcount+total, links)  
    else:
        truecase[] = () if '''
    fp_l.write('\t'.join( [lemma, page, langlink[page] if page in langlink else '___', str(total), str(links),'\n']))
    #print ('\t'.join( [lemma, page, langlink[page] if page in langlink else '___', str(total), str(links)]))
fp_l.close() 

In [64]:
print(redirect['acid'])

ACiD Productions


In [48]:
!head -35 en.link.redirect.txt

'hood	'hood	___	3	Counter({"[['hood]]": 3})	
'hood	Ghetto	聚集區	2	Counter({"[[ghetto|'hood]]": 1, "[[Ghetto|'hood]]": 1})	
'hood	Neighbourhood	近鄰社區	1	Counter({"[[Neighborhood|'hood]]": 1})	
'Hood	Ghetto	聚集區	1	Counter({"[[Ghetto|'Hood]]": 1})	
'Hood	'Hood	___	1	Counter({"[['Hood]]": 1})	
'Hood	'Hood (film)	___	1	Counter({"[['Hood (film)|'Hood]]": 1})	
'Hood	'Hood (1998 film)	___	1	Counter({"[['Hood (1998 film)|'Hood]]": 1})	
.22	.22 caliber	___	83	Counter({'[[.22]]': 66, '[[.22 caliber|.22]]': 16, '[[.22 calibre|.22]]': 1})	
.22-caliber	.22 caliber	___	3	Counter({'[[.22|.22-caliber]]': 3})	
.22-caliber	Caliber	口徑	1	Counter({'[[Caliber|.22-caliber]]': 1})	
.22_caliber	.22_caliber	___	1	Counter({'[[.22_caliber]]': 1})	
.38-caliber	.38 caliber	___	3	Counter({'[[.38 caliber|.38-caliber]]': 2, '[[.38 Caliber|.38-caliber]]': 1})	
.45-caliber	11 mm caliber	___	3	Counter({'[[.45 caliber|.45-caliber]]': 3})	
0	0,	0	869	Counter({'[[0 (number)|0]]': 867, '[[zero (number)|0]]': 2})	
0	0s	0年代	49	Count

In [14]:
!grep -P '^party\t' en.link.redirect.txt

party	Political party	政党	2306	Counter({'[[Political party|party]]': 1068, '[[political party|party]]': 705, '[[political parties|parties]]': 244, '[[political party|parties]]': 216, '[[Political party|parties]]': 64, '[[Political parties|parties]]': 3, '[[Political Party|parties]]': 2, '[[Party (politics)|party]]': 2, '[[political parties|party]]': 1, '[[Political parties|party]]': 1})	
party	Party	派對	868	Counter({'[[party]]': 684, '[[parties]]': 86, '[[party|parties]]': 62, '[[Party|parties]]': 36})	
party	Nazi Party	纳粹党	258	Counter({'[[Nazi Party|party]]': 240, '[[Nazi party|party]]': 18})	
party	Party (law)	___	222	Counter({'[[party (law)|party]]': 66, '[[Party (law)|party]]': 62, '[[Party (law)|parties]]': 52, '[[party (law)|parties]]': 42})	
party	Party (role-playing games)	___	163	Counter({'[[Party (role-playing games)|party]]': 118, '[[Party (role playing games)|party]]': 20, '[[Party (role-playing games)|parties]]': 14, '[[party (role-playing games)|party]]': 5, '[[Party (role 

<font face="微軟正黑體" size=5>6. 找出標題和文章中的連結所對應的中文</font><br>

<font face="微軟正黑體" size=5>output1：en.link.graph.gz; 形式：  &lt;page_title&gt; \n anchors </font><br>

In [50]:
!bzgrep -P -o '(<title>[^\<]+|\[\[[^]#:][^]:]*\]\])' enwiki-latest-pages-articles-multistream.xml.bz2 | sed 's/#.*|/|/g' | sed 's/#.*]]/]]/g' | gzip > en.link.graph.gz

<font face="微軟正黑體" size=5>output2：en.related.zh.link.txt; 形式：  page_title \t zh_title \t zh_titles_in_article</font><br>

In [58]:
fp_r = open("en.related.zh.txt", "w")

for line in gzip.open('en.link.graph.gz','rt').readlines():
    line = line.strip()
    if line.startswith("<title>"):
        entry = line[len("<title>"):]
        fp_r.write("\n"+entry)
        fp_r.write("\t"+langlink.get(redirect_link(entry), '___')+"\t")
    else:
        link = line.split('|')[0][2:] if '|' in line else line[2:-2]
        zh = langlink.get(redirect_link(link), '')
        if len(zh) > 0:
            fp_r.write(zh + ",")
fp_r.close()

In [71]:
%%bash
cat en.link.redirect.txt | cut -d $'\t' -f2 | sed 's/^ *//' | sort | uniq > pages.txt

join -t $'\t' pages.txt <(sort en.related.zh.txt) > en.related.zh.link.txt


In [78]:
! sed -i 's/,$//' en.related.zh.link.txt
! sed -i '/^$/d' en.related.zh.link.txt

In [79]:
!grep '^Party' en.related.zh.link.txt

Party	派對	肯特郡,第二次世界大战,款待,社會化,休閒活動,节日,食物,饮料,音乐,舞蹈,娛樂,啤酒,Wine,蒸馏酒,名譽,超级碗,圣帕特里克十字,餐馆,英式酒吧,夜總會,酒吧,懺悔星期二,和平条约,生日蛋糕,生日,西方文化,气球,生日蛋糕,祝你生日快乐,成年禮,气球,晚餐,鸡尾酒,Wine,自助餐,小礼服,饮料,小菜,甜品,飯局,公园,園林,郊遊,燒烤,英國君主政體,巴士底日,鸡尾酒,小礼服,茶,下午茶,骨瓷,三明治,香蕉,蛋糕,小圆面包,曲奇,白金汉宫,白宮,婚宴,演講,學術研討會,自助餐,美術館,博物馆,策展人,国宴,民族舞蹈,亚述人,芝加哥,畢業舞會,方塊舞,舞会,社交舞,白領結,酒,面具,懺悔星期二,英格兰,美国,圣诞节,聖誕頌歌,我們祝你聖誕快樂,聖誕節的十二天,铃儿响叮当,聖誕老人進城來,圣善夜,流行樂團,柏林,House (音樂),Techno,迪斯科,唱片騎師,銳舞,House (音樂),Techno,工业音乐,MDMA,迷幻药物,北美洲,英国,澳大利亚,合法飲酒年齡,酒,未成年人,噪音,地下酒吧,飲,篝火,畢業舞會,畢業,友情,約會,性交,学校,慈善组织,企业,小彼得·勃鲁盖尔,婚宴,远藤周作,剧本,音樂劇,歌剧,电影,电视节目,婚礼,猶太教成人禮,婚礼,圣诞节,國際友誼日,懺悔星期二,澳大利亞日,墨爾本盃,加拿大日,維多利亞日,巴士底日,德國統一日,宗教改革日,玫瑰星期一,圣诞节,排燈節,侯丽节,纳吾肉孜节,圣帕特里克十字,逾越節,光明节,普珥節,五月五日节,怀唐伊日,巴基斯坦独立日,霍格莫內,仲夏節,篝火之夜,美国独立日,超级碗,感恩节 (美国),乌拉圭历史,圣诞节,復活節,懺悔星期二,圣帕特里克十字,古尔邦节,開齋節,光明节,逾越節,普珥節,燃燒人,燃燒人,滿月派對,Lollapalooza,白晝之夜,胡士托音樂節,加拿大,聚会游戏,聚会游戏,圖版遊戲,LAN party,局域网,直销,特百惠,選舉,睡衣派對,青少年,托加,丹佛,小龍蝦派對,节日,銳舞,群交,春假,婚宴
Party (disambiguation)	___	第三党,派對,婚礼,政党,Xbox Live,独唱团,主机保护区域,The Miracle,第三党,游戏开发者
Party (law)	___	個人,法律主體,法律,原告,被告,證人,訴訟,原告,被告,絕對權
P

In [15]:
!grep "'Birthday_Party'" en.redirect.txt

22820514	'The_Birthday_Party_(album)'	'Birthday_Party'
38744420	'Birthday_party_(disambiguation)'	'Birthday_Party'
39176408	'Birthday_Party_(disambiguation)'	'Birthday_Party'
536391	'The_Birthday_Party'	'Birthday_Party'


In [17]:
!grep "Birthday_Party" en.page.id.txt

10113483	'The_Birthday_Party_(film)'
10813796	'The_Birthday_Party_(Dynasty)'
13867333	'Happy_Birthday_(Birthday_Party_song)'
14523090	'The_Birthday_Party_(The_Birthday_Party_album)'
16611852	'The_Birthday_Party_(Motorhead_video)'
17245702	'Happy_Birthday_(1980_Birthday_Party_song)'
191226	'The_Birthday_Party_(band)'
19682483	'Birthday_Party_(TV_series)'
21348335	'The_Birthday_Party_(Birthday_Party_album)'
21387145	'The_Birthday_Party_(novel)'
21563008	'A_Collection_(The_Birthday_Party_album)'
2190558	'At_Your_Birthday_Party'
22063607	'The_Great_San_Bernardino_Birthday_Party_&_Other_Excursions'
22113697	'Ninetieth_Birthday_Party'
22820514	'The_Birthday_Party_(album)'
23912734	'Birthday_Party_Bash'
24679243	'A_Birthday_Party_for_Julia_Child:_Compliments_to_the_Chef'
25680361	'Arco_Birthday_Party'
2767923	'The_Birthday_Party_(play)'
2853567	'Birthday_Party_(band)'
34248731	'25th_Birthday_Party'
36922776	'The_Great_San_Bernardino_Birthday_Party_and_Other_Excursions'
3775383	'Eeyores_Birthd

In [16]:
!grep "Birthday_party" en.page.id.txt

30520572	'Birthday_party_(event)'
30521682	'Birthday_party'
38744420	'Birthday_party_(disambiguation)'


In [21]:
!grep '^Birthday' en.link.redirect.txt # anchor Birthday would be removed because it is capitalized.

In [19]:
!zgrep "#" en.link.wn.gz

In [21]:
ll = "aaa#bbb"
if "#" in ll:
    ll = ll.split("#")[0]
print(ll)

aaa
